Import the necessary packages

In [1]:
import cv2
import numpy as np
import os
import platform
if platform.uname().system == 'Windows':
    os.chdir('C:/Users/ralt6/source/repos/FactorioBot/')
else:
    os.chdir('/home/ryan/Projects/FactorioBot/')

Declare image paths

In [2]:
main_img_path = 'src/images/test_image_1.jpg'
enemy_img_path = 'src/images/ti1_dead_enemy_vertical.jpg'

Load images and reduce size by half

In [3]:
main_img = cv2.imread(main_img_path, cv2.IMREAD_REDUCED_COLOR_2)
enemy_img = cv2.imread(enemy_img_path, cv2.IMREAD_REDUCED_COLOR_2)

Display image sizes

In [4]:
h, w = main_img.shape[:2]
print("Height = {}, width = {}".format(h,w))
h,w = enemy_img.shape[:2]
print("Height = {}, width = {}".format(h,w))

Height = 450, width = 800
Height = 39, width = 34


Extract R,B,G values from a particular pixel

In [5]:
b, g, r = main_img[100,100]
print('R: {}, G: {}, B: {}'.format(r,g,b))

R: 47, G: 18, B: 0


Extract a particular region of interest

In [6]:
roi = main_img[100:400, 100:400]
def display_image(img):
    cv2.imshow('Section', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Convert images to grayscale. Set pixel values above 120 to 255 and pixel at 120 or below to 0. Display image

In [7]:
main_gs = cv2.cvtColor(main_img, cv2.COLOR_BGR2GRAY)
enemy_gs = cv2.cvtColor(enemy_img, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(main_gs, 120, 255, cv2.THRESH_BINARY)
display_image(thresh1)

Compare enemy image against test image


In [8]:
result = cv2.matchTemplate(main_img, enemy_img, cv2.TM_CCOEFF_NORMED)
display_image(result)

Get the location of the max item observed

In [9]:
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
print("Image located at Location: {} with value: {}".format(max_loc, max_val))

Image located at Location: (528, 219) with value: 0.9939625263214111


Draw rectangle around found dead enemy in test image

In [15]:
eh,ew = enemy_img.shape[:2]
top_left = max_loc
bottom_right = (max_loc[0] + ew, max_loc[1] + eh)
result = cv2.rectangle(main_img, top_left, bottom_right, color=(0,255,0), thickness=2, lineType=cv2.LINE_4)
display_image(result)
cv2.imwrite('src/images/test_image_1_ti1_result.jpg', result)

True

Gather list of results of nearly matching images above a certain threshold

In [35]:
main_img = cv2.imread(main_img_path, cv2.IMREAD_REDUCED_COLOR_2)
result = cv2.matchTemplate(main_img, enemy_img, cv2.TM_CCOEFF_NORMED)
def find_above_threshold(result, threshold):
    locations = np.where(result >= threshold)
    return list(zip(*locations[::-1]))
    #for i in range(len(result)):
    #    for j in range(len(result[i])):
    #        if result[i][j] > threshold:
    #            print('Print threshold found at {} with value {}'.format((i,j), result[i][j]))
print(find_above_threshold(result, 0.4))

[(527, 214), (528, 214), (529, 214), (526, 215), (527, 215), (528, 215), (526, 217), (527, 217), (528, 217), (529, 217), (530, 217), (531, 217), (532, 217), (525, 218), (526, 218), (527, 218), (528, 218), (529, 218), (530, 218), (531, 218), (532, 218), (525, 219), (526, 219), (527, 219), (528, 219), (529, 219), (530, 219), (531, 219), (525, 220), (526, 220), (527, 220), (528, 220), (529, 220), (530, 220), (531, 220), (525, 221), (526, 221), (527, 221), (528, 221), (529, 221), (530, 221), (528, 223), (529, 223), (530, 223), (528, 224), (529, 224), (429, 248), (430, 248), (429, 249), (430, 249), (429, 250), (430, 250), (427, 255), (428, 255), (429, 255), (430, 255), (428, 256), (429, 256), (388, 405), (389, 405), (388, 406), (389, 406)]


Draw rectangles for all found locations

In [56]:
def draw_rectangles(img, locations, dimension, name):
    for i in locations:
        top_left = i
        bottom_right = (top_left[0] + dimension[0], top_left[1] + dimension[1])
        cv2.rectangle(img, top_left, bottom_right, color=(0,255,0), thickness=2, lineType=cv2.LINE_4)
    cv2.imwrite('src/images/{}.jpg'.format(name), img)

enemy_img = cv2.imread(enemy_img_path)
methods = [(cv2.TM_SQDIFF, 'SQDIFF'), (cv2.TM_SQDIFF_NORMED, 'SQDIFF_NORMED'), (cv2.TM_CCORR, 'CCORR'), 
    (cv2.TM_CCORR_NORMED, 'CCORR_NORMED'), (cv2.TM_CCOEFF, 'CCOEFF'), (cv2.TM_CCOEFF_NORMED, 'CCOEFF_NORMED')]
for method in methods:
    main_img = cv2.imread(main_img_path)
    result = cv2.matchTemplate(main_img, enemy_img, method[0])
    locations = find_above_threshold(result, 0.98)
    draw_rectangles(main_img, locations, enemy_img.shape[:2], 'ti1_evert_test_result2_{}'.format(method[1]))